# Shadowing Transform
This notebook demonstrates the shadowing transform. Shadowing corresponds to real-world path blockages and channel effects that cause signal path losses to deviate significantly from unobstructed free-space path loss between transmitter and receiver. Though these effects vary greatly, they are commonly modeled as following a log-normal response (Refer to: https://www.gaussianwaves.com/2013/09/log-distance-path-loss-or-log-normal-shadowing-model/), as parameterized by the log-normal distribution mean and standard deviation.

In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.datasets.dataset_metadata import DatasetMetadata
from torchsig.signals.builders.tone import ToneSignalBuilder
import torchsig.transforms.functional as F
from torchsig.utils.dsp import (
    TorchSigComplexDataType
)

import numpy as np
from scipy import stats

%matplotlib inline
import matplotlib.pyplot as plt
from scipy.signal import spectrogram

Generator function for QPSK test signal.

In [ ]:
def generate_tone_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband tone Signal.

    Args:
        num_iq_samples (int, optional): Length of sample. Defaults to 128.
        scale (int, optional): Scale factor for normalized signal data. Defaults to 1.0.

    Returns:
        signal: generated Signal.

    """
    rng = np.random.default_rng(42)
    sample_rate = 10e6
    
    md = DatasetMetadata(
        num_iq_samples_dataset = num_iq_samples,
        fft_size = 4,
        impairment_level = 0,
        sample_rate = sample_rate,
        num_signals_max = 1,
        num_signals_min = 1,
        num_signals_distribution = [1.0],
        snr_db_min = 10.0,
        snr_db_max = 10.0,
        signal_duration_min = 1.00*num_iq_samples/sample_rate,
        signal_duration_max = 1.00*num_iq_samples/sample_rate,
        signal_bandwidth_min = sample_rate/4,
        signal_bandwidth_max = sample_rate/4,
        signal_center_freq_min = 0.0,
        signal_center_freq_max = 0.0, 
        transforms = [],
        target_transforms = [],
        class_list = ['tone'],
        class_distribution = [1.0],
        num_samples = 1,
        seed = 1234
    )
    builder = ToneSignalBuilder(
        dataset_metadata = md, 
        class_name = 'tone',
        seed = 1234
    )
    signal = builder.build()

    # normalize, then scale data   
    signal.data = F.normalize(
        data = signal.data,
        norm_order = 2,
        flatten = False
    )
    signal.data = np.multiply(signal.data, scale).astype(TorchSigComplexDataType)

    return signal

Generate test input data.

In [ ]:
rng = np.random.default_rng(42)

# signal parameters
N = 10000
sampling_rate = 4.0
fc = 0.2 # center frequency

# generate, frequency shift, and power scale the input tone signal
tone_bb_data = generate_tone_signal(num_iq_samples = N, scale = 1.0).data 
tone_data = tone_bb_data * np.exp(2j * np.pi * fc * np.arange(N) / sampling_rate)
tone_power = np.mean(np.abs(tone_data) ** 2)

Test log-normal shadowing power distribution with multiple transform calls.

In [ ]:
# functional check
n_iterations = 1000
results = [
    10*np.log10(np.mean(np.abs(
        F.shadowing(
            data = tone_bb_data,
            mean_db = 4.0,
            sigma_db = 2.0,
            rng = rng
        )**2)
    ))
    for _ in  range(n_iterations)
]
results_array = np.array(results)

# Shapiro-Wilk test for normality
stat, p_value = stats.shapiro(results_array)

print(f"Shapiro-Wilk test statistic: {stat:.3f}, p-value: {p_value:.3f}")
if p_value > 0.05:
    print("The distribution is likely normal")
else:
    print("The distribution is likely not normal")

Plot output power distribution histogram.

In [ ]:
fig = plt.figure(figsize=(6,6))
plt.hist(results_array);
plt.title('Shadowing: Power Distribution');
plt.xlabel('Mean Signal Power [dB]');
plt.ylabel('Frequency');